In [1]:
import pandas as pd
import scipy.stats as stats
import rpy2.robjects as robjects

from statsmodels.stats.proportion import proportions_ztest

In [2]:
df = pd.read_csv("./data/homepage-experiment-data.csv")
df.head()

,Day,Control Cookies,Control Downloads,Control Licenses,Experiment Cookies,Experiment Downloads,Experiment Licenses
0,1,1764,246,1,1850,339,3
1,2,1541,234,2,1590,281,2
2,3,1457,240,1,1515,274,1
3,4,1587,224,1,1541,284,2
4,5,1606,253,2,1643,292,3


# Checking the Invariant Metric

What is the p-value for the test on the number of cookies assigned to each group?


- $H_{0}: \text{The two groups are balanced.}$
- $H_{\alpha}: \text{The two groups are imblanced.}$

- $\alpha = 0.05$

In [3]:
n1 = df["Control Cookies"].sum()  # 46851
n2 = df["Experiment Cookies"].sum()  # 47346
n = n1 + n2  # 94197

In [4]:
# One-proportion z-test
zstat, pval = proportions_ztest(
    count=n1, nobs=n, value=.5, alternative="two-sided", prop_var=False
)
zstat, pval

(-1.6128451019747376, 0.10677816462098283)

In [5]:
# R
# Define an R code snippet to run
r_code = f'''
prop.test(
    x=c({n1}),
    n=c({n}),
    p=.5,
    alternative="two.sided",
    # Note that, by default, the function prop.test() used the Yates continuity correction, 
    # which is really important if either the expected successes or failures is < 5. 
    # If you don’t want the correction, use the additional argument correct = FALSE in prop.test() function. 
    # The default value is TRUE. (This option must be set to FALSE to make the test mathematically equivalent to 
    # the uncorrected z-test of a proportion.)
    correct=FALSE
)
'''

# Run the R code
print(robjects.r(r_code))


	1-sample proportions test without continuity correction

data:  c(46851) out of c(94197), null probability 0.5
X-squared = 2.6012, df = 1, p-value = 0.1068
alternative hypothesis: true p is not equal to 0.5
95 percent confidence interval:
 0.4941797 0.5005655
sample estimates:
        p 
0.4973725 




---

# Checking the Evaluation Metric I

What is the p-value for the test on the download rate between groups?

- $H_{0}: \text{P(Download rate | control group)} = \text{P(Download rate | treatment group)}$
- $H_{\alpha}: \text{P(Download rate | control group)} < \text{P(Download rate | treatment group)}$

- $\alpha = 0.05$

In [6]:
# Two-sample pooled z-test for proportion by proportions_ztest
dl1 = df["Control Downloads"].sum()  # 7554
dl2 = df["Experiment Downloads"].sum()  # 8548

# difference
d0 = 0  # assume there is no difference of click-throught rate between two groups
count = [dl1, dl2]
nobs = [n1, n2]

zstat, pval = proportions_ztest(
    count,
    nobs,
    value=d0,  # null hypothesis
    alternative="smaller",
    prop_var=False,  # pooled
)
zstat, pval

(-7.870833726066236, 1.7614279636728079e-15)

In [7]:
# R
# Define an R code snippet to run
r_code = f'''
prop.test(
    x=c({dl1}, {dl2}),
    n=c({n1}, {n2}),
    p=NULL,
    alternative="less",
    conf.level=.975,
    # Note that, by default, the function prop.test() used the Yates continuity correction, 
    # which is really important if either the expected successes or failures is < 5. 
    # If you don’t want the correction, use the additional argument correct = FALSE in prop.test() function. 
    # The default value is TRUE. (This option must be set to FALSE to make the test mathematically equivalent to 
    # the uncorrected z-test of a proportion.)
    correct=FALSE
)
'''

# Run the R code
print(robjects.r(r_code))


	2-sample test for equality of proportions without continuity correction

data:  c(7554, 8548) out of c(46851, 47346)
X-squared = 61.95, df = 1, p-value = 1.761e-15
alternative hypothesis: less
97.5 percent confidence interval:
 -1.00000000 -0.01450323
sample estimates:
   prop 1    prop 2 
0.1612346 0.1805432 




---

## Checking the Evaluation Metric II

What is the p-value for the test on the license purchasing rate between groups?

- $H_{0}: \text{P(License purchase rate | control group)} = \text{P(License purchase rate | treatment group)}$
- $H_{\alpha}: \text{P(License purchase rate | control group)} < \text{P(License purchase rate | treatment group)}$

- $\alpha = 0.05$

In [8]:
# Two-sample pooled z-test for proportion by proportions_ztest
# get the first 21 days
n1 = df.loc[:20, "Control Cookies"].sum()  # 33758
n2 = df.loc[:20, "Experiment Cookies"].sum()  # 34338
# license
lic1 = df["Control Licenses"].sum()  # 710
lic2 = df["Experiment Licenses"].sum()  # 732

# difference
d0 = 0  # assume there is no difference of click-throught rate between two groups
count = [lic1, lic2]
nobs = [n1, n2]

zstat, pval = proportions_ztest(
    count,
    nobs,
    value=d0,  # null hypothesis
    alternative="smaller",
    prop_var=False,  # pooled
)
zstat, pval

(-0.2586750111658684, 0.3979430008399871)

In [9]:
# R
# Define an R code snippet to run
r_code = f'''
prop.test(
    x=c({lic1}, {lic2}),
    n=c({n1}, {n2}),
    p=NULL,
    alternative="less",
    conf.level=.975,
    # Note that, by default, the function prop.test() used the Yates continuity correction, 
    # which is really important if either the expected successes or failures is < 5. 
    # If you don’t want the correction, use the additional argument correct = FALSE in prop.test() function. 
    # The default value is TRUE. (This option must be set to FALSE to make the test mathematically equivalent to 
    # the uncorrected z-test of a proportion.)
    correct=FALSE
)
'''

# Run the R code
print(robjects.r(r_code))


	2-sample test for equality of proportions without continuity correction

data:  c(710, 732) out of c(33758, 34338)
X-squared = 0.066913, df = 1, p-value = 0.3979
alternative hypothesis: less
97.5 percent confidence interval:
 -1.000000000  0.001877193
sample estimates:
    prop 1     prop 2 
0.02103205 0.02131749 


